In [ ]:
import pandas as pd

import plotly.express as pt



from dash import Dash, dcc, html, Input, Output, ctx, dash_table

df = pd.read_csv('C:/Users/1/Downloads/Bakery.csv', delimiter = ',')

df.head()

days = {0: 'Monday',
       1: 'Thuesday',
       2: 'Wensday',
       3: 'Thursday',
       4: 'Friday',
       5: 'Saturday',
       6: 'Sunday'}

month = {'01':'Janauary',
'02':'February',
'03':'March',
'04':'April',
'05':'May',
'06':'June',
'07': 'July',
'08':'August',
'09':'September',
'10':'October',
'11':'November',
'12':'December'}

df['DateTime'] = pd.to_datetime(df['DateTime'])
df['year'] = df['DateTime'].dt.year
df['day'] = df['DateTime'].dt.dayofweek
df['day'] = df['day'].apply(lambda x: days[x])
df['month'] = df['DateTime'].dt.month

month_dict = month.items()

fin = {i: j[-1] for i,j in zip(range(1,13), month_dict)}

df['month'] = df['month'].apply(lambda x: fin[x])

df_for_hours = df.drop_duplicates(subset = ['TransactionNo'])

df_for_hours['hour'] = df_for_hours['DateTime'].dt.hour
foga2 = pt.histogram(df_for_hours[(df_for_hours['hour'] >= 7) & (df_for_hours['hour'] <= 23)], x = 'hour', labels = {'hour': 'Hour', 'count': 'Count'}, title = 'Which hours are the most intense')



df_for_hours['Time_prev_tran'] = [0] + list(df_for_hours['DateTime'])[:-1]

df_for_hours.loc[0,'Time_prev_tran'] = df_for_hours.loc[0,'DateTime']

df_for_hours['Time_difference_min'] = (df_for_hours['DateTime'] - df_for_hours['Time_prev_tran']).dt.total_seconds() / 60

df_for_hours['Time_difference_min'] = df_for_hours['Time_difference_min'].round(2)

df_for_hours['Time_difference_day'] = (df_for_hours['DateTime'] - df_for_hours['Time_prev_tran']).dt.total_seconds() / 60 / 60 / 24

df_for_hours['Time_difference_day'] = df_for_hours['Time_difference_day'].round(2)

df_for_hours = df_for_hours[(df_for_hours['Time_difference_min'] > 0) & (df_for_hours['Time_difference_day'] < 1)]



prod_of_day = df[['Items','day']].groupby(['day','Items'])['Items'].agg(['count'])


prod_of_day.reset_index(inplace = True)

prod_of_day.rename(columns = {'count': 'number'}, inplace = True)



fig3 = pt.histogram(df, x = 'month', labels = {'month': 'Month'}, title = 'In which months the shop has more clients')

hour_group = df_for_hours[['year','day','Daypart','Time_difference_min']].groupby(['year','day','Daypart'])['Time_difference_min'].agg(['mean','median', 'std'])

hour_group.reset_index(inplace = True)

count_items = df.value_counts('Items').to_frame()
count_items.reset_index(inplace = True)



count_items.rename(columns = {0: 'Number'}, inplace = True)

days.values()

foga = pt.bar(count_items.loc[:8,:].sample(frac = 1), x = 'Items', y = 'Number', labels = {'Items': 'Items'},  title = 'Most popular products')

app1 = Dash()

app1.layout = html.Div(children =[
            html.H1(children = "The analysis of shop's sales intensity",
                    style = {'textAlign': 'center'}),
            html.Div([dcc.Graph(id = 'popular what', figure = foga),], style = {'width': '33%', 'display':'inline-block', 'vertical-align': 'top'}),
            html.Div([dcc.Graph(id = 'know_what', figure = foga2)], style = {'width': '33%', 'display': 'inline-block', 'vertical-align': 'top',}),
            html.Div([
            dcc.Graph(id = 'Comparison'),
            dcc.RadioItems(options = list(days.values()), value = 'Monday', id = 'days', inline = True),
            dcc.RadioItems(options = [2016,2017], value = 2016, id = 'years'),
            ], style={'width': '33%', 'display': 'inline-block', 'vertical-align': 'top'}),
            html.Div([dcc.Graph(id = 'months', figure = fig3)])])



@app1.callback(
Output('Comparison', 'figure'),
Input('years', 'value'),
Input('days', 'value'))
def update_graph(year,daya):
    ala = hour_group[(hour_group['year'] == year) & (hour_group['day'] == daya) & (hour_group['Daypart'] != 'Night')]
    fig = pt.bar(ala, x = 'Daypart', y = 'mean', labels = {'Daypart': 'Daypart', 'mean': 'Mean_interval(mins)'}, title = 'Activeness of our customers in different days')
    
    return fig



if __name__ == '__main__':
    app1.run_server(debug = False, port = 2000)




